In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import StackingClassifier

In [2]:
# Load the dataset
df = pd.read_csv('cancer_classification_train.csv')
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,benign_0__mal_1
0,17.990,10.38,122.80,1001.0,0.11840,0.27760,0.300100,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.66560,0.71190,0.26540,0.4601,0.11890,0
1,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.086900,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.18660,0.24160,0.18600,0.2750,0.08902,0
2,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.197400,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.42450,0.45040,0.24300,0.3613,0.08758,0
3,11.420,20.38,77.58,386.1,0.14250,0.28390,0.241400,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.86630,0.68690,0.25750,0.6638,0.17300,0
4,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.198000,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.20500,0.40000,0.16250,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,10.160,19.59,64.73,311.7,0.10030,0.07504,0.005025,0.01116,0.1791,0.06331,...,22.88,67.88,347.3,0.1265,0.12000,0.01005,0.02232,0.2262,0.06742,1
557,9.423,27.88,59.26,271.3,0.08123,0.04971,0.000000,0.00000,0.1742,0.06059,...,34.24,66.50,330.6,0.1073,0.07158,0.00000,0.00000,0.2475,0.06969,1
558,14.590,22.68,96.39,657.1,0.08473,0.13300,0.102900,0.03736,0.1454,0.06147,...,27.27,105.90,733.5,0.1026,0.31710,0.36620,0.11050,0.2258,0.08004,1
559,11.510,23.93,74.52,403.5,0.09261,0.10210,0.111200,0.04105,0.1388,0.06570,...,37.16,82.28,474.2,0.1298,0.25170,0.36300,0.09653,0.2112,0.08732,1


In [3]:
# Separate features and target
X = df.drop(columns=['benign_0__mal_1'])
y = df['benign_0__mal_1']

In [4]:
# Split the data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [5]:
# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
# Initialize base models for stacking
log_reg = LogisticRegression(max_iter=1000)
rf = RandomForestClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)

In [7]:
bagging_clf = BaggingClassifier(estimator=RandomForestClassifier(random_state=42),
                                n_estimators=50, random_state=42)


In [8]:

# Build a Stacking Classifier
stacking_clf = StackingClassifier(
    estimators=[('rf', rf), ('gb', gb)],  # Using Random Forest and Gradient Boosting as base models
    final_estimator=log_reg  # Logistic Regression as the meta-model
)

In [9]:
# Train the Bagging model
bagging_clf.fit(X_train_scaled, y_train)

# Train the Stacking model
stacking_clf.fit(X_train_scaled, y_train)

StackingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                               ('gb',
                                GradientBoostingClassifier(random_state=42))],
                   final_estimator=LogisticRegression(max_iter=1000))

In [10]:
# Evaluate the Bagging model
y_pred_bagging = bagging_clf.predict(X_test_scaled)
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
classification_report_bagging = classification_report(y_test, y_pred_bagging)

In [11]:

# Evaluate the Stacking model
y_pred_stacking = stacking_clf.predict(X_test_scaled)
accuracy_stacking = accuracy_score(y_test, y_pred_stacking)
classification_report_stacking = classification_report(y_test, y_pred_stacking)

In [12]:
# Output results
print(f'Bagging Model Accuracy: {accuracy_bagging}')
print(f'Bagging Model Classification Report:\n{classification_report_bagging}')

Bagging Model Accuracy: 0.9557522123893806
Bagging Model Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.91      0.94        43
           1       0.95      0.99      0.97        70

    accuracy                           0.96       113
   macro avg       0.96      0.95      0.95       113
weighted avg       0.96      0.96      0.96       113



In [13]:
print(f'Stacking Model Accuracy: {accuracy_stacking}')
print(f'Stacking Model Classification Report:\n{classification_report_stacking}')

Stacking Model Accuracy: 0.9292035398230089
Stacking Model Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.86      0.90        43
           1       0.92      0.97      0.94        70

    accuracy                           0.93       113
   macro avg       0.93      0.92      0.92       113
weighted avg       0.93      0.93      0.93       113



Interpretation:

Accuracy: Both models have high accuracy, which means they correctly classified a large percentage of cancerous and non-cancerous cases.

Precision and Recall: High precision and recall indicate that the models performed well in identifying both benign and malignant cases.

F1-score: The F1-score balances precision and recall, indicating the robustness of the model in terms of both true positives and false positives.


In [15]:
# Load the dataset
df_N = pd.read_csv('validation_1.csv')
df_N

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,benign_0__mal_1
0,11.20,29.37,70.67,386.0,0.07449,0.03558,0.00000,0.00000,0.1060,0.05502,...,38.30,75.19,439.6,0.09267,0.05494,0.0000,0.0000,0.1566,0.05905,1
1,15.22,30.62,103.40,716.9,0.10480,0.20870,0.25500,0.09429,0.2128,0.07152,...,42.79,128.70,915.0,0.14170,0.79170,1.1700,0.2356,0.4089,0.14090,0
2,20.92,25.09,143.00,1347.0,0.10990,0.22360,0.31740,0.14740,0.2149,0.06879,...,29.41,179.10,1819.0,0.14070,0.41860,0.6599,0.2542,0.2929,0.09873,0
3,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
4,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
5,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
6,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0
7,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,0.1587,0.05884,...,30.37,59.16,268.6,0.08996,0.06444,0.0000,0.0000,0.2871,0.07039,1


In [16]:
# Separate features and target
validation_data= df_N.drop(columns=['benign_0__mal_1'])

In [17]:
validation_data

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,11.20,29.37,70.67,386.0,0.07449,0.03558,0.00000,0.00000,0.1060,0.05502,...,11.920,38.30,75.19,439.6,0.09267,0.05494,0.0000,0.0000,0.1566,0.05905
1,15.22,30.62,103.40,716.9,0.10480,0.20870,0.25500,0.09429,0.2128,0.07152,...,17.520,42.79,128.70,915.0,0.14170,0.79170,1.1700,0.2356,0.4089,0.14090
2,20.92,25.09,143.00,1347.0,0.10990,0.22360,0.31740,0.14740,0.2149,0.06879,...,24.290,29.41,179.10,1819.0,0.14070,0.41860,0.6599,0.2542,0.2929,0.09873
3,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
4,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
5,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
6,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400
7,7.76,24.54,47.92,181.0,0.05263,0.04362,0.00000,0.00000,0.1587,0.05884,...,9.456,30.37,59.16,268.6,0.08996,0.06444,0.0000,0.0000,0.2871,0.07039


In [18]:
# Assuming you have a DataFrame 'validation_data' containing your 8 validation samples.
# Load the validation data (if it's from a file) - replace with the actual file or data
# validation_data = pd.read_csv('path_to_validation_data.csv')

# Example data for demonstration (replace with your actual validation data)
# validation_data should be a DataFrame with the same columns (features) as your training data
# Standardize the validation data using the scaler that was fit on the training data
validation_data_scaled = scaler.transform(validation_data)

# Use the trained models to predict whether each sample is benign (0) or malignant (1)
stacking_predictions = stacking_clf.predict(validation_data_scaled)
bagging_predictions = bagging_clf.predict(validation_data_scaled)

# Map the predictions to 'Benign' or 'Malignant'
stacking_results = ['Malignant' if pred == 1 else 'Benign' for pred in stacking_predictions]
bagging_results = ['Malignant' if pred == 1 else 'Benign' for pred in bagging_predictions]

# Print results
print("Stacking Model Predictions:")
for i, result in enumerate(stacking_results):
    print(f"Sample {i+1}: {result}")

print("\nBagging Model Predictions:")
for i, result in enumerate(bagging_results):
    print(f"Sample {i+1}: {result}")


Stacking Model Predictions:
Sample 1: Malignant
Sample 2: Benign
Sample 3: Benign
Sample 4: Benign
Sample 5: Benign
Sample 6: Benign
Sample 7: Benign
Sample 8: Malignant

Bagging Model Predictions:
Sample 1: Malignant
Sample 2: Benign
Sample 3: Benign
Sample 4: Benign
Sample 5: Benign
Sample 6: Benign
Sample 7: Benign
Sample 8: Malignant
